This notebook is for converting a Keras Model to a TensorFlow Model

## Setup

In [1]:
%%bash

echo 'Train:'
TRAIN_DIR=../../../tmp/trt_end_to_end/train
find ${TRAIN_DIR} -type f
echo
echo 'Convert:'
CONVERT_TF_DIR=../../../tmp/trt_end_to_end/convert/tf
find ${CONVERT_TF_DIR} -type f

Train:
../../../tmp/trt_end_to_end/train/keras_model_basic_epoch001_2019-09-02T19:22.h5

Convert:
../../../tmp/trt_end_to_end/convert/tf/tf_model_basic_epoch001_2019-09-02T19:22.data-00000-of-00001
../../../tmp/trt_end_to_end/convert/tf/tf_model_basic_epoch001_2019-09-02T19:22.meta
../../../tmp/trt_end_to_end/convert/tf/tf_model_basic_epoch001_2019-09-02T19:22.index
../../../tmp/trt_end_to_end/convert/tf/checkpoint


In [2]:
import sys
sys.path.append('../../..')

## Parameters

In [3]:
import os
_TMP_DIR = '../../../tmp'
_TRT_END_TO_END_DIR = os.path.join(_TMP_DIR, 'trt_end_to_end')
_TRAIN_DIR = os.path.join(_TRT_END_TO_END_DIR, 'train')
_CONVERT_TF_DIR = os.path.join(_TRT_END_TO_END_DIR, 'convert/tf')
_NAME = 'basic'
_EPOCH = 1
_TIME = '2019-09-02T19:22'

## Set learning phase 0

In [4]:
# import the needed libraries
import tensorflow as tf
print(tf.__name__, tf.__version__, sep='-')
tf.keras.backend.set_learning_phase(0) #use this if we have batch norm layer in our network

/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:519: FutureWarning: Passing (type, 1) or 

tensorflow-1.14.0


/usr/local/lib/python3.6/dist-packages/tensorboard/compat/tensorflow_stub/dtypes.py:541: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/usr/local/lib/python3.6/dist-packages/tensorboard/compat/tensorflow_stub/dtypes.py:542: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/usr/local/lib/python3.6/dist-packages/tensorboard/compat/tensorflow_stub/dtypes.py:543: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/usr/local/lib/python3.6/dist-packages/tensorboard/compat/tensorflow_stub/dtypes.py:544: FutureWa

## Load keras model

In [5]:
from src.utils.trt_end_to_end_constants import TRAIN_FILE_FORMAT
import os

model_filename = TRAIN_FILE_FORMAT % (_NAME, _EPOCH, _TIME)

keras_model = tf.keras.models.load_model(os.path.join(_TRAIN_DIR, model_filename))

W0902 19:25:01.820852 140510579111744 deprecation.py:506] From /usr/local/lib/python3.6/dist-packages/tensorflow/python/ops/init_ops.py:97: calling GlorotUniform.__init__ (from tensorflow.python.ops.init_ops) with dtype is deprecated and will be removed in a future version.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
W0902 19:25:01.821735 140510579111744 deprecation.py:506] From /usr/local/lib/python3.6/dist-packages/tensorflow/python/ops/init_ops.py:1251: calling VarianceScaling.__init__ (from tensorflow.python.ops.init_ops) with dtype is deprecated and will be removed in a future version.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
W0902 19:25:01.822882 140510579111744 deprecation.py:506] From /usr/local/lib/python3.6/dist-packages/tensorflow/python/ops/init_ops.py:97: calling Zeros.__init__ (from tensorflow.python.ops.init_ops) with dtyp

## Save tensorflow model

In [6]:
%%bash

CONVERT_TF_DIR=../../../tmp/trt_end_to_end/convert/tf
mkdir -p ${CONVERT_TF_DIR} && rm -f ./${CONVERT_TF_DIR}/*
find ${CONVERT_TF_DIR} -type f

In [7]:
from src.utils.trt_end_to_end_constants import CONVERT_TF_FILE_FORMAT

file_prefix = CONVERT_TF_FILE_FORMAT % (_NAME, _EPOCH ,_TIME)
save_to = os.path.join(_CONVERT_TF_DIR, file_prefix)

# save the model to Tensorflow model
saver = tf.train.Saver()
sess = tf.keras.backend.get_session()
save_path = saver.save(sess, save_to)

print("Keras model is successfully converted to TF model in "+_CONVERT_TF_DIR)

Keras model is successfully converted to TF model in ../../../tmp/trt_end_to_end/convert/tf


In [8]:
%%bash

CONVERT_TF_DIR=../../../tmp/trt_end_to_end/convert/tf
find ${CONVERT_TF_DIR} -type f

../../../tmp/trt_end_to_end/convert/tf/tf_model_basic_epoch001_2019-09-02T19:22.data-00000-of-00001
../../../tmp/trt_end_to_end/convert/tf/tf_model_basic_epoch001_2019-09-02T19:22.meta
../../../tmp/trt_end_to_end/convert/tf/tf_model_basic_epoch001_2019-09-02T19:22.index
../../../tmp/trt_end_to_end/convert/tf/checkpoint
